## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
###Drop Nan Values
df=df.dropna()


In [8]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [9]:
df.shape

(18285, 5)

In [10]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [12]:
## Get the Dependent features
y=df['label']

In [13]:
X.shape

(18285, 4)

In [14]:
y.shape

(18285,)

In [15]:
import tensorflow as tf

In [16]:
tf.__version__

'2.14.0'

In [39]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dense

In [18]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [19]:
messages=X.copy()

In [20]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [21]:
messages.shape

(18285, 4)

In [22]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [23]:
messages.reset_index(inplace=True)

In [24]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [25]:
import nltk
import re
from nltk.corpus import stopwords

In [26]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pranayrajsarap/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
messages['title']

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                        Why the Truth Might Get You Fired
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
18280    Rapper T.I.: Trump a ’Poster Child For White S...
18281    N.F.L. Playoffs: Schedule, Matchups and Odds -...
18282    Macy’s Is Said to Receive Takeover Approach by...
18283    NATO, Russia To Hold Parallel Exercises In Bal...
18284                            What Keeps the F-35 Alive
Name: title, Length: 18285, dtype: object

In [28]:
sentences = messages['title']

In [29]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(sentences)):
    review = re.sub('[^a-zA-Z]', ' ', sentences[i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [30]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [31]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [32]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[3984, 4106, 4538, 3317, 1814, 1988, 273, 1257, 3375, 181],
 [1359, 2489, 4420, 1618, 3122, 3729, 2497],
 [2904, 2280, 2832, 3170],
 [4486, 1625, 3969, 4803, 3916, 1161],
 [2091, 3122, 3146, 148, 3222, 4191, 3122, 258, 1231, 3448],
 [3094,
  4223,
  2923,
  3207,
  473,
  2666,
  1434,
  2006,
  1077,
  2308,
  4492,
  3695,
  635,
  895,
  2497],
 [4134, 1218, 2523, 305, 2891, 1700, 2778, 3933, 4331, 3852, 2824],
 [1485, 3037, 3087, 2921, 2905, 3089, 2666, 1822, 4331, 3852, 2824],
 [850, 1622, 3084, 1278, 445, 1019, 2634, 1377, 2666, 2566],
 [3300, 734, 3509, 3627, 3447, 3326, 4133, 3292],
 [1586, 93, 884, 1958, 4226, 2283, 3004, 1399, 4830, 2444, 3226],
 [4803, 2557, 1814, 1019, 2666, 2905],
 [2592, 2683, 2618, 4772, 1295, 1620, 728, 1502, 409],
 [2884, 711, 3448, 4136, 104, 3418, 2480, 4331, 3852, 2824],
 [867, 1293, 1717, 993, 818, 4331, 3852, 2824],
 [1642, 4885, 4025, 4666, 4172, 703, 1566, 2343, 1830, 1410],
 [4664, 2189, 2489],
 [908, 2453, 4147, 2764, 2666, 1919, 4988, 2497],

In [33]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [34]:
onehot_repr[1]

[1359, 2489, 4420, 1618, 3122, 3729, 2497]

### Embedding Representation

In [35]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[3984 4106 4538 ...    0    0    0]
 [1359 2489 4420 ...    0    0    0]
 [2904 2280 2832 ...    0    0    0]
 ...
 [ 220 1455 1333 ...    0    0    0]
 [4691 2905 2283 ...    0    0    0]
 [3299  747  547 ...    0    0    0]]


In [36]:
embedded_docs[1]

array([1359, 2489, 4420, 1618, 3122, 3729, 2497,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [37]:
embedded_docs[0]

array([3984, 4106, 4538, 3317, 1814, 1988,  273, 1257, 3375,  181,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [40]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [41]:
len(embedded_docs),y.shape

(18285, (18285,))

In [42]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [43]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [45]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 4s 14ms/step - loss: 0.3081 - accuracy: 0.8534 - val_loss: 0.1887 - val_accuracy: 0.9238
Epoch 2/10
192/192 [==============================] - 3s 13ms/step - loss: 0.1204 - accuracy: 0.9549 - val_loss: 0.2035 - val_accuracy: 0.9223
Epoch 3/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0718 - accuracy: 0.9772 - val_loss: 0.2253 - val_accuracy: 0.9173
Epoch 4/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0480 - accuracy: 0.9856 - val_loss: 0.2602 - val_accuracy: 0.9130
Epoch 5/10
192/192 [==============================] - 4s 19ms/step - loss: 0.0297 - accuracy: 0.9917 - val_loss: 0.3460 - val_accuracy: 0.9158
Epoch 6/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0182 - accuracy: 0.9945 - val_loss: 0.3597 - val_accuracy: 0.9094
Epoch 7/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0167 - accuracy: 0.9952 - val_loss: 0.3669 - val_accuracy: 0.9084

### Adding Dropout

In [46]:
# from tensorflow.keras.layers import Dropout
# ## Creating model
# embedding_vector_features=40
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(Dropout(0.3))
# model.add(LSTM(100))
# model.add(Dropout(0.3))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [47]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 2ms/step


In [48]:
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve

In [49]:
from sklearn.metrics import confusion_matrix

In [50]:
confusion_matrix(y_test,y_pred)

array([[3135,  284],
       [ 253, 2363]])

In [51]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9110190555095278

In [52]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.92      0.92      3419
           1       0.89      0.90      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

